# Optimal Trader Joe's Grocery List

In [262]:
pip install pulp --quiet


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [263]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

## Loading Data

In [264]:
data = pd.read_csv('nutrition_prices.csv')
data

/var/folders/zp/sq6bzsp91fz1sl3107mgk5gc0000gn/T/ipykernel_44785/2670820770.py:1: DtypeWarning: Columns (2,10,11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('nutrition_prices.csv')


,Unnamed: 0.1,Unnamed: 0,serving_size,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,total_carbohydrates,dietary_fiber,sugars,protein,vitamin_d,calcium,iron,potassium,item,item_title,retail_price
0,0,0,2 tbsp. (28g),70.0,7.0,5.0,0.0,20.0,70.0,1.0,0,0.5,2.0,0.0,52.0,0.0,0,"""Stracciatella"" Burata Filling",Burrata Filling,4.49
1,225,225,0.8 cup (170g),110.0,0.0,0.0,0.0,10.0,75.0,7.0,0,5,17.0,0.0,190.0,0.0,240,"0% Greek Yogurt, Nonfat, Plain",Nonfat Plain Greek Yogurt,0.99
2,818,818,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0,15,11.0,0.8,130.0,0.4,188,0% Milkfat Greek Nonfat Yogurt,Greek Spanakopita,4.49
3,1407,1407,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0,15,12.0,0.8,130.0,0.4,188,0% Milkfat Strawberry Greek Nonfat Yogurt,Sparkling Strawberry Juice,3.99
4,1993,1993,1 1-inch cube (28g),120.0,10.0,7.0,0.0,30.0,270.0,0.0,0,0,8.0,0.0,260.0,0.0,0,"1,000 Day Gouda Cheese",1000 Day Gouda Cheese,12.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59019,619083,9 pieces (30g),120.0,1.5,0.5,0.0,5.0,80.0,27.0,1.0,11.0,0.5,NaN,NaN,NaN,20.0,Churro Bites,Mac and Cheese Bites,3.99,NaN
59020,619084,9 pieces (30g),120.0,1.5,0.5,0.0,5.0,80.0,27.0,1.0,11.0,0.5,NaN,NaN,NaN,20.0,Churro Bites,Mac and Cheese Bites,3.99,NaN
59021,619085,9 pieces (30g),120.0,1.5,0.5,0.0,5.0,80.0,27.0,1.0,11.0,0.5,NaN,NaN,NaN,20.0,Churro Bites,Mac and Cheese Bites,3.99,NaN
59022,619086,9 pieces (30g),120.0,1.5,0.5,0.0,5.0,80.0,27.0,1.0,11.0,0.5,NaN,NaN,NaN,20.0,Churro Bites,Mac and Cheese Bites,3.99,NaN


In [265]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59024 entries, 0 to 59023
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0.1         59024 non-null  int64  
 1   Unnamed: 0           27772 non-null  object 
 2   serving_size         27702 non-null  object 
 3   calories             58953 non-null  float64
 4   total_fat            58430 non-null  float64
 5   saturated_fat        58415 non-null  float64
 6   trans_fat            58412 non-null  float64
 7   cholesterol          59004 non-null  float64
 8   sodium               59021 non-null  float64
 9   total_carbohydrates  58429 non-null  float64
 10  dietary_fiber        58415 non-null  object 
 11  sugars               59013 non-null  object 
 12  protein              22707 non-null  float64
 13  vitamin_d            23205 non-null  float64
 14  calcium              23204 non-null  float64
 15  iron                 32473 non-null 

## Cleaning Data

In [266]:
columns_to_drop = ['Unnamed: 0.1', 'Unnamed: 0']
nutrition_data_cleaned = data.drop(columns=columns_to_drop)

In [267]:
numeric_columns = ['dietary_fiber', 'sugars', 'potassium']
for col in numeric_columns:
    nutrition_data_cleaned[col] = pd.to_numeric(nutrition_data_cleaned[col], errors='coerce')


In [268]:
critical_columns = ['calories', 'protein', 'retail_price', 'calcium', 'iron', 'potassium']
nutrition_data_cleaned = nutrition_data_cleaned.dropna(subset=critical_columns)

In [269]:
nutrition_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 0 to 175
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   serving_size         85 non-null     object 
 1   calories             85 non-null     float64
 2   total_fat            85 non-null     float64
 3   saturated_fat        84 non-null     float64
 4   trans_fat            83 non-null     float64
 5   cholesterol          84 non-null     float64
 6   sodium               85 non-null     float64
 7   total_carbohydrates  85 non-null     float64
 8   dietary_fiber        85 non-null     float64
 9   sugars               85 non-null     float64
 10  protein              85 non-null     float64
 11  vitamin_d            85 non-null     float64
 12  calcium              85 non-null     float64
 13  iron                 85 non-null     float64
 14  potassium            85 non-null     float64
 15  item                 85 non-null     object 
 

In [270]:
columns_to_check = ['total_fat', 'total_carbohydrates', 'vitamin_d', 'sodium', 'dietary_fiber', 'sugars']
for col in columns_to_check:
    nutrition_data_cleaned[col].fillna(0, inplace=True)

/var/folders/zp/sq6bzsp91fz1sl3107mgk5gc0000gn/T/ipykernel_44785/2889761469.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nutrition_data_cleaned[col].fillna(0, inplace=True)


In [271]:
nutrition_data_cleaned = nutrition_data_cleaned[
    (nutrition_data_cleaned['calories'] > 0) &
    (nutrition_data_cleaned['protein'] > 0) &
    (nutrition_data_cleaned['retail_price'] > 0)
]

In [272]:
nutrition_data_cleaned

,serving_size,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,total_carbohydrates,dietary_fiber,sugars,protein,vitamin_d,calcium,iron,potassium,item,item_title,retail_price
0,2 tbsp. (28g),70.0,7.0,5.0,0.0,20.0,70.0,1.0,0.0,0.5,2.0,0.0,52.0,0.0,0.0,"""Stracciatella"" Burata Filling",Burrata Filling,4.49
1,0.8 cup (170g),110.0,0.0,0.0,0.0,10.0,75.0,7.0,0.0,5.0,17.0,0.0,190.0,0.0,240.0,"0% Greek Yogurt, Nonfat, Plain",Nonfat Plain Greek Yogurt,0.99
2,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0.0,15.0,11.0,0.8,130.0,0.4,188.0,0% Milkfat Greek Nonfat Yogurt,Greek Spanakopita,4.49
3,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0.0,15.0,12.0,0.8,130.0,0.4,188.0,0% Milkfat Strawberry Greek Nonfat Yogurt,Sparkling Strawberry Juice,3.99
4,1 1-inch cube (28g),120.0,10.0,7.0,0.0,30.0,270.0,0.0,0.0,0.0,8.0,0.0,260.0,0.0,0.0,"1,000 Day Gouda Cheese",1000 Day Gouda Cheese,12.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,1 pack (71g),380.0,29.0,10.0,0.0,0.0,120.0,23.0,6.0,14.0,16.0,0.0,100.0,6.7,450.0,"Amped-Up Almonds, Chocolatey Coated Almonds",Amped-Up Almonds,0.01
169,0.8 cup (51g),220.0,10.0,4.0,0.0,0.0,15.0,29.0,4.0,8.0,5.0,0.0,26.0,1.8,95.0,Ancient Grains & Nuts Granola,Organic Yellow Lentil & Brown Rice Spaghetti,3.49
173,1 cup (255g),310.0,16.0,5.0,1.0,45.0,760.0,25.0,6.0,5.0,18.0,0.0,78.0,5.4,705.0,"Angus Beef Chili, with Pinto Beans",Chevre with Honey,3.29
174,1 patty (151g),470.0,39.0,17.0,1.5,95.0,65.0,0.0,0.0,0.0,28.0,0.4,26.0,2.7,376.0,Angus C.B.P. Ground Beef Patties,Angus C.B.P. Ground Beef Patties,9.99


## Optimization Model

#### Model

In [273]:
model = LpProblem("Optimal_Grocery_List", LpMinimize)

#### Binary Decision Variables

In [274]:
x = {i: LpVariable(f"x_{i}", cat="Binary") for i in nutrition_data_cleaned.index}

#### Objecvtive Function to Minimize Cost

In [275]:
model += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index), "Total Cost"

#### Constraints

In [276]:
constraints = {
    "calories": 2200,         
    "protein": 60,            
    "fat_max": 80,             
    "carbohydrate": 250,       
    "sodium_max": 3000,        
    "fiber": 20,              
    "sugar_max": 60,          
    "cholesterol_max": 300,   
    "saturated_fat_max": 20,  
    "vitamin_d": 10,          
    "budget": 250              
}

In [277]:
# Constraint 1: Calories 
model += lpSum(nutrition_data_cleaned.loc[i, 'calories'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["calories"], "Calorie_Constraint"
# Constraint 2: Protein
model += lpSum(nutrition_data_cleaned.loc[i, 'protein'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["protein"], "Protein_Constraint"
# Constraint 3: Fat
model += lpSum(nutrition_data_cleaned.loc[i, 'total_fat'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["fat_max"], "Fat_Constraint"
# Constraint 4: Carbohydrates
model += lpSum(nutrition_data_cleaned.loc[i, 'total_carbohydrates'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["carbohydrate"], "Carbohydrate_Constraint"
# Constraint 5: Sodium
model += lpSum(nutrition_data_cleaned.loc[i, 'sodium'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["sodium_max"], "Sodium_Constraint"
# Constraint 6: Fiber
model += lpSum(nutrition_data_cleaned.loc[i, 'dietary_fiber'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["fiber"], "Fiber_Constraint"
# Constraint 7: Sugar
model += lpSum(nutrition_data_cleaned.loc[i, 'sugars'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["sugar_max"], "Sugar_Constraint"
# Constraint 8: Cholesterol
model += lpSum(nutrition_data_cleaned.loc[i, 'cholesterol'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["cholesterol_max"], "Cholesterol_Constraint"
# Constraint 9: Saturated Fat
model += lpSum(nutrition_data_cleaned.loc[i, 'saturated_fat'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["saturated_fat_max"], "Saturated_Fat_Constraint"
# Constraint 10: Vitamin D
model += lpSum(nutrition_data_cleaned.loc[i, 'vitamin_d'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["vitamin_d"], "Vitamin_D_Constraint"
# Constraint 11: Budget
model += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["budget"], "Budget_Constraint"

#### Model Solver

In [278]:
model.solve()
print("Model Status:", LpStatus[model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josesalerno/Documents/BU/Semester/Fall-24/BA885/traderjoe/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/zp/sq6bzsp91fz1sl3107mgk5gc0000gn/T/43887582c99d40d085133fe2ab92b8c4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/zp/sq6bzsp91fz1sl3107mgk5gc0000gn/T/43887582c99d40d085133fe2ab92b8c4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 867 RHS
At line 879 BOUNDS
At line 954 ENDATA
Problem MODEL has 11 rows, 74 columns and 628 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29.4853 - 0.00 seconds
Cgl0004I processed model has 11 rows, 72 columns (72 integer (70 of which binary)) and 616 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 5 integers unsatisfied sum - 0.832219


In [281]:
selected_items = [nutrition_data_cleaned.loc[i, 'item_title'] for i in nutrition_data_cleaned.index if x[i].value() == 1]
print("Selected Items:")
for item in selected_items:
    print(item)

Selected Items:
Nonfat Plain Greek Yogurt
Whole Wheat Sliced Bread
3 Seed Beet Crackers
Sweet Potato
Strawberry Waffles
4 Cheese Ravioli
Sprouted Wheat Sourdough Bread
Wild Skipjack Tuna
Solid White Albacore Tuna, No Salt Added
Almond Butter Chia Overnight Oats
Gluten Free Ginger Turmeric Granola
Everything But The Bagel Seasoned Potato Chips
Sweet Pull Apart Aloha Rolls


In [282]:
total_cost = sum(nutrition_data_cleaned.loc[i, 'retail_price'] for i in nutrition_data_cleaned.index if x[i].value() == 1)
print(f"Total Cost: ${total_cost:.2f}")

Total Cost: $31.87
